In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/Users/carlalopez/documents/Lie_Symmetry_Analysis_Agorithm/src')

In [4]:
from symmetries.analysis.lie_symmetry_analysis import point_symmetries
from symmetries.utils.symbolic import sym_det_eqn

from symmetries.objects.general_form import GeneralForm
import sympy as sp
from sympy import Derivative as D

# One equation of Hall MHD 

In [2]:
x, y, z, t = sp.symbols('x y z t')

phi = sp.Function('phi')(x, y, z, t)
psi = sp.Function('psi')(x, y, z, t)
b = sp.Function('b')(x, y, z, t)
w = sp.Function('w')(x, y, z, t)

sigma, gamma = sp.symbols('sigma gamma')

list_indep = [x, y, z, t]
list_dep = [phi, psi, b, w]
list_cte = [sigma, gamma]

F =  D(b, t) + D(b, x)*D(phi, y) - D(b, y)*D(phi, x) + sigma*(D(psi, x)*(D(psi, x, x, y) + D(psi, y, y, y))-
        D(psi, y)*(D(psi, x, x, x) + D(psi, y, y, x))) + D(psi, x)*D(w, y) - D(psi, y)*D(w, x) - gamma*(D(b,x,x) + D(b, y, y))

F_rules_array = {D(psi, x, x, y): 
        - (sigma*D(psi, x))**(-1)*(D(b, t) + D(b, x)*D(phi, y) - D(b, y)*D(phi, x) 
        + D(psi, x)*D(w, y) - D(psi, y)*D(w, x) 
        - gamma*(D(b,x,x) + D(b, y, y)))
        +(D(psi, x))**(-1)*D(psi, y)*(D(psi, x, x, x) + D(psi, y, y, x))
        - D(psi, y, y, y)}

order = 3
F.expand()

-gamma*Derivative(b(x, y, z, t), (x, 2)) - gamma*Derivative(b(x, y, z, t), (y, 2)) + sigma*Derivative(psi(x, y, z, t), x)*Derivative(psi(x, y, z, t), (y, 3)) + sigma*Derivative(psi(x, y, z, t), x)*Derivative(psi(x, y, z, t), (x, 2), y) - sigma*Derivative(psi(x, y, z, t), (x, 3))*Derivative(psi(x, y, z, t), y) - sigma*Derivative(psi(x, y, z, t), y)*Derivative(psi(x, y, z, t), (y, 2), x) + Derivative(b(x, y, z, t), t) + Derivative(b(x, y, z, t), x)*Derivative(phi(x, y, z, t), y) - Derivative(b(x, y, z, t), y)*Derivative(phi(x, y, z, t), x) + Derivative(psi(x, y, z, t), x)*Derivative(w(x, y, z, t), y) - Derivative(psi(x, y, z, t), y)*Derivative(w(x, y, z, t), x)

# Heat Equation

In [6]:
x, t, k = sp.symbols('x t k')
u = sp.Function('u')(x, t)
independent_variables = [x, t]
dependent_variables = [u]
constants = [k]
F =  D(u,t) - k*D(u,x,x)
F_rules_array = {D(u,x,x):1/k*D(u,t)}
order = 2
F.expand()

-k*Derivative(u(x, t), (x, 2)) + Derivative(u(x, t), t)

In [5]:
model, system_of_equations = point_symmetries(F, order, F_rules_array, independent_variables, dependent_variables, constants)

In [7]:
general_form = GeneralForm(model, system_of_equations)
general_form.print_matrix()

general form: {'xi': {'x': [x, t, u(x, t)], 't': [x, t, u(x, t)]}, 'eta': {'u': [x, t, u(x, t)]}}
already deleted: {}


Matrix([
[0,                                 Eq(xi^(t)_u_u, 0)],
[1,                                 Eq(xi^(x)_u_u, 0)],
[2,                                   Eq(xi^(t)_u, 0)],
[3,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[4,                Eq(2*k*xi^(t)_x_u + 2*xi^(x)_u, 0)],
[5,                                   Eq(xi^(t)_x, 0)],
[6,       Eq(k*xi^(t)_x_x - xi^(t)_t + 2*xi^(x)_x, 0)],
[7, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[8,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

In [8]:
general_form.find_first_derivative_equals_0()

deleting xit u(x, t)


In [11]:
general_form.print_matrix()

general form: {'xi': {'x': [x, t, u(x, t)], 't': [t]}, 'eta': {'u': [x, t, u(x, t)]}}
already deleted: {'xit': [u(x, t), x]}


Matrix([
[0,                                 Eq(xi^(t)_u_u, 0)],
[1,                                 Eq(xi^(x)_u_u, 0)],
[2,                                   Eq(xi^(t)_u, 0)],
[3,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[4,                Eq(2*k*xi^(t)_x_u + 2*xi^(x)_u, 0)],
[5,                                   Eq(xi^(t)_x, 0)],
[6,       Eq(k*xi^(t)_x_x - xi^(t)_t + 2*xi^(x)_x, 0)],
[7, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[8,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

In [14]:
general_form.find_deleted_items_in_equations()
general_form.print_matrix()

general form: {'xi': {'x': [x, t, u(x, t)], 't': [t]}, 'eta': {'u': [x, t, u(x, t)]}}
already deleted: {'xit': [u(x, t), x]}


Matrix([
[0,                                 Eq(xi^(t)_u_u, 0)],
[1,                                 Eq(xi^(x)_u_u, 0)],
[2,                                   Eq(xi^(t)_u, 0)],
[3,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[4,                                   Eq(xi^(x)_u, 0)],
[5,                                   Eq(xi^(t)_x, 0)],
[6,                     Eq(-xi^(t)_t + 2*xi^(x)_x, 0)],
[7, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[8,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

In [15]:
general_form.find_first_derivative_equals_0()

deleting xix u(x, t)


In [16]:
general_form.print_matrix()

general form: {'xi': {'x': [x, t], 't': [t]}, 'eta': {'u': [x, t, u(x, t)]}}
already deleted: {'xit': [u(x, t), x], 'xix': [u(x, t)]}


Matrix([
[0,                                 Eq(xi^(t)_u_u, 0)],
[1,                                 Eq(xi^(x)_u_u, 0)],
[2,                                   Eq(xi^(t)_u, 0)],
[3,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[4,                                   Eq(xi^(x)_u, 0)],
[5,                                   Eq(xi^(t)_x, 0)],
[6,                     Eq(-xi^(t)_t + 2*xi^(x)_x, 0)],
[7, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[8,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

In [17]:
general_form.find_deleted_items_in_equations()

In [18]:
general_form.print_matrix()

general form: {'xi': {'x': [x, t], 't': [t]}, 'eta': {'u': [x, t, u(x, t)]}}
already deleted: {'xit': [u(x, t), x], 'xix': [u(x, t)]}


Matrix([
[0,                                 Eq(xi^(t)_u_u, 0)],
[1,                                 Eq(xi^(x)_u_u, 0)],
[2,                                   Eq(xi^(t)_u, 0)],
[3,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[4,                                   Eq(xi^(x)_u, 0)],
[5,                                   Eq(xi^(t)_x, 0)],
[6,                     Eq(-xi^(t)_t + 2*xi^(x)_x, 0)],
[7, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[8,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

In [19]:
general_form.delete_second_derivatives()

found u(x, t) in xit eq 0
found u(x, t) in xix eq 1


In [31]:
general_form.find_deleted_items_in_equations()

found u(x, t) in xix eq 3
found u(x, t) in xix eq 3


In [33]:
general_form.print_matrix()

general form: {'xi': {'x': [x, t], 't': [t]}, 'eta': {'u': [x, t, u(x, t)]}}
already deleted: {'xit': [u(x, t), x], 'xix': [u(x, t)]}


Matrix([
[0,                                   Eq(xi^(t)_u, 0)],
[1,                                Eq(eta^(u)_u_u, 0)],
[2,                                   Eq(xi^(x)_u, 0)],
[3,                                   Eq(xi^(t)_x, 0)],
[4,                     Eq(-xi^(t)_t + 2*xi^(x)_x, 0)],
[5, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[6,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

# Blasius Equation

In [70]:
x = sp.symbols('x')
y = sp.Function('y')(x)
list_indep = [x]
list_dep = [y]
list_cte = []
F =  D(y,x,x,x) + y*D(y,x,x)
F_rules_array = {D(y,x,x,x):-y*D(y,x,x)}
order = 3
point_symmetries(F, order, F_rules_array, list_indep, list_dep, list_cte)

(<symmetries.objects.system.System at 0x7f8db13fb100>,
 <symmetries.objects.determining_equations.DeterminingEquations at 0x7f8dc04c05e0>)

In [14]:
F.expand()

y(x)*Derivative(y(x), (x, 2)) + Derivative(y(x), (x, 3))

# Wave Equation

In [71]:
x, t, k = sp.symbols('x t k')
u = sp.Function('u')(x, t)
list_indep = [x, t]
list_dep = [u]
list_var = list_indep + list_dep
list_cte = [k]
list_all = list_cte + list_indep + list_dep
F =  D(u,t) - k*D(u,x,x)
F_rules_array = {D(u,x,x):1/k*D(u,t)}
order = 2
F.expand()

-k*Derivative(u(x, t), (x, 2)) + Derivative(u(x, t), t)

In [73]:
model, system_of_equations = point_symmetries(F, order, F_rules_array, list_indep, list_dep, list_cte)

In [74]:
general_form = GeneralForm(model, system_of_equations)
general_form.print_matrix()

general form: {'xi': {'x': [x, t, u(x, t)], 't': [x, t, u(x, t)]}, 'eta': {'u': [x, t, u(x, t)]}}
already deleted: {}


Matrix([
[0,                                 Eq(xi^(t)_u_u, 0)],
[1,                                 Eq(xi^(x)_u_u, 0)],
[2,                                   Eq(xi^(t)_u, 0)],
[3,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[4,                Eq(2*k*xi^(t)_x_u + 2*xi^(x)_u, 0)],
[5,                                   Eq(xi^(t)_x, 0)],
[6,       Eq(k*xi^(t)_x_x - xi^(t)_t + 2*xi^(x)_x, 0)],
[7, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[8,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

In [77]:
general_form.find_first_derivative_equals_0()

In [79]:
general_form.delete_second_derivatives()

found u(x, t) in xit eq 0
found x in xit eq 6


In [85]:
general_form.print_matrix()

general form: {'xi': {'x': [x, t, u(x, t)], 't': [t]}, 'eta': {'u': [x, t, u(x, t)]}}
already deleted: {'xit': [u(x, t), x]}


Matrix([
[0,                                 Eq(xi^(x)_u_u, 0)],
[1,                                   Eq(xi^(t)_u, 0)],
[2,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[3,                                   Eq(xi^(x)_u, 0)],
[4,                                   Eq(xi^(t)_x, 0)],
[5,                     Eq(-xi^(t)_t + 2*xi^(x)_x, 0)],
[6, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[7,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

In [82]:
general_form.find_deleted_items_in_equations()

found x in xit eq 4
found x in xit eq 4
found x in xit eq 6


In [84]:
general_form.print_matrix()

general form: {'xi': {'x': [x, t, u(x, t)], 't': [t]}, 'eta': {'u': [x, t, u(x, t)]}}
already deleted: {'xit': [u(x, t), x]}


Matrix([
[0,                                 Eq(xi^(x)_u_u, 0)],
[1,                                   Eq(xi^(t)_u, 0)],
[2,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[3,                                   Eq(xi^(x)_u, 0)],
[4,                                   Eq(xi^(t)_x, 0)],
[5,                     Eq(-xi^(t)_t + 2*xi^(x)_x, 0)],
[6, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[7,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

# Schrödinger Equation

In [7]:
x, t, k, i = sp.symbols('x t k i')
u = sp.Function('u')(x, t)
list_indep = [x, t]
list_dep = [u]
list_cte = [k, i]
F =  -D(u,x,x) +x**2*u-i*k*D(u,t)
F_rules_array = {D(u,x,x):x**2-i*k*D(u,t)}
order = 2
F.expand()
point_symmetries(F, order, F_rules_array, list_indep, list_dep, list_cte)

Matrix([
[1,                                                        Eq(i*k*xi^(t)_t - 2*i*k*xi^(x)_x + x**2*xi^(t), 0)],
[2,                                           Eq(-2*eta^(u)_x_u + i*k*xi^(x)_t + x**2*xi^(x) + xi^(x)_x_x, 0)],
[3, Eq(eta^(u)*x**2 - eta^(u)_t*i*k - eta^(u)_u*x**2 - eta^(u)_x_x + 2*x**2*xi^(x)_x + 2*x*xi^(x)*u(x, t), 0)],
[4,                                                                                         Eq(xi^(t)_u_u, 0)],
[5,                                                                                         Eq(xi^(x)_u_u, 0)],
[6,                                                                                           Eq(xi^(t)_u, 0)],
[7,                                                                                           Eq(xi^(t)_x, 0)],
[8,                                                                                           Eq(xi^(x)_u, 0)],
[9,                                                                                        Eq(e

# HNLS Equation

In [4]:
# Real
z, t, g1, g2, l, s = sp.symbols('z t gamma_1 gamma_2 lambda_0 sigma')
u = sp.Function('u')(z, t)
v = sp.Function('v')(z, t)
list_indep = [z, t]
list_dep = [u, v]
list_cte = [g1, g2, l, s]
F =  D(u,t) + D(v,z,z) + l*(v*u**2+v**3)+s*D(u,z,z,z)+\
    g1*D(u,z)*(u**2+v**2)+2*g2*u*(u*D(u,z)+v*D(v,z))
F_rules_array = {D(u,z,z,z): (-1/s)*(D(u,t) + D(v,z,z) + l*(v*u**2+v**3)+
    g1*D(u,z)*(u**2+v**2)+2*g2*u*(u*D(u,z)+v*D(v,z)))}
order = 3
F.expand()
point_symmetries(F, order, F_rules_array, list_indep, list_dep, list_cte)

NameError: name 'N_indep' is not defined

In [ ]:
# Imaginaria
z, t, g1, g2, l, s = sp.symbols('z t gamma_1 gamma_2 lambda_0 sigma')
u = sp.Function('u')(z, t)
v = sp.Function('v')(z, t)
list_indep = [z, t]
list_dep = [u, v]
list_cte = [g1, g2, l, s]
F =-D(v,t) + D(u,z,z) + l*(v**2*u+u**3)-s*D(v,z,z,z)-\
    g1*D(v,z)*(u**2+v**2)-2*g2*v*(u*D(u,z)+v*D(v,z))
F_rules_array = {D(v,z,z,z): 1/s*(-D(v,t) + D(u,z,z) + l*(v**2*u+u**3)-\
    g1*D(v,z)*(u**2+v**2)-2*g2*v*(u*D(u,z)+v*D(v,z)))}
order = 3
F.expand()
point_symmetries(F, order, F_rules_array, list_indep, list_dep, list_cte)